# Seminario sobre Python - Pandas

## Los datos: Encuesta Financiera de las Familias 2014 (BdE)

El seminario se va a basar en los datos de la __[EFF](https://www.bde.es/bde/es/areas/estadis/Otras_estadistic/Encuesta_Financi/)__. Se trata precisamente de un panel de unas 6.000 familias que se viene realizando desde el año 2.002 con el fin de obtener información directa sobre las condiciones financieras de las familias españolas. Permite la elaboración de estudios sobre las decisiones de inversión y financiación de las familias españolas y sobre su situación patrimonial, que resultan relevantes para el mejor conocimiento de nuestra economía y el adecuado diseño de las políticas públicas.

<img src="https://www.caixabank.es/deployedfiles/particulares/Estaticos/Imagenes/Segmentos/FotobannerFamilydestacadook-2.jpg" alt="pandas" title="family" width="400" />

Para este ejercicio vamos a centrarnos en la encuesta de 2014.

***

## Objetivos

- Familiarizarse con el proceso de datos, datos descriptivos en tablas y gráficos con Pandas
- Aplicar técnicas de Machine Learning (ML)


## Ejercicio

Vamos a analizar una campaña de márquetin de préstamos para la compra de automóviles. Primero se pide clasificar las familias según su propensión a compra de automóviles del último año. En segundo lugar, proponer una financiación personalizada que cubra la totalidad del valor del vehículo. 

Para saber qué familias han comprado un automóvil, tenemos la variable "p2_74" que representa el gasto total realizado para su adquisición. Cuando esta variable es positiva, significa ha adquirido un vehículo. Esta variable se encuentra en la tabla "otras_secciones_2014_imp1" (tabla A a partir de ahora). La tabla databol5 la llamaremos tabla B.

Para poder estimar la propensión se cuenta con el siguiente diccionario de variables:
- renta familiar (B.renthog13_eur14)
- edad del cabeza de familia (CF) (A.p1_2d_1)
- nivel de estudios del CF (B.neducdom): toma valores 1 a 3; inferior a bachillerato, bachillerato, estudios universitarios
- situación laboral del CF (B.nsitlabdom): toma valores 1 a 4; empleado por cuenta ajena, empleado por cuenta propia, jubilado, otro tipo de inactivo o parado 
- número de miembros del hogar (B.np1): toma valores 1 a 5; uno, dos, tres, cuatro, cinco o más
- niembros trabajando (B.nnumadtrab): toma valores 0 a 3; ninguno, uno, dos, tres o más

En resumen, lo que se pide es:
1. Estimar la propensión a compra de un automóvil en el último año
  - Opción 1:__[statsmodels glm](http://www.statsmodels.org/dev/examples/notebooks/generated/glm_formula.html)__
  - Opción 2:__[sklearn.ensemble.GradientBoostingClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html#sklearn.ensemble.GradientBoostingClassifier)__

2. Financiación a ofrecer: Estimar el percentil 90% de la distribución de precios de compra.
  - Opcion 1: __[statsmodels quantile_regression](http://www.statsmodels.org/dev/examples/notebooks/generated/quantile_regression.html)__
  - Opcion 1:__[sklearn.ensemble.GradientBoostingRegressor con loss='huber' o loss='quantile'.](http://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_quantile.html#sphx-glr-auto-examples-ensemble-plot-gradient-boosting-quantile-py)__



La carpeta de trabajo es: 
> "L:\16630\Big Data Lab\3. Formacio\L. Formacion Interna\python\seminario1"

El cuestionario se puede en: 
> "eff/cuestionario_2014.pdf"

Las variables calculadar en databol estan en: 
> "eff/definitions_2014.doc"


### Carga de librerías

In [0]:
from __future__ import print_function
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import ensemble
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.regression.quantile_regression import QuantReg
from statsmodels.compat import lmap
from scipy import stats
import patsy
import xgboost as xgb
import tensorflow as tf

import matplotlib.pyplot as plt   # librerias graficas 
import math  # funciones matemáticas
import re    # expresiones regulares 
import os    # funciones de sistema operativo
#os.chdir("L:/16630/Big Data Lab/3. Formacio/L. Formacion Interna/python/seminario1")
#os.chdir("C:/DL/pandas")

# habilitamos graficos para notebooks de Jupyter
%matplotlib inline

# Establecemos el límite de mostrar columnas a 200
pd.set_option("display.max_columns",200)

In [0]:
otras_secciones_2014_imp1 = pd.read_stata("https://cdn.rawgit.com/griu/pandasLearning/a7cb43c8/secciones_2014.dta")
databol5 = pd.read_stata("https://cdn.rawgit.com/griu/pandasLearning/a7cb43c8/databol.dta")

### Validamos la carga

In [0]:
print(otras_secciones_2014_imp1.shape)
print(databol5.shape)

# filas x columnas

In [0]:
otras_secciones_2014_imp1[["p2_74","p1_2d_1"]].head() 

In [0]:
databol5[["renthog13_eur14","neducdom","nsitlabdom","np1","nnumadtrab"]].head() 

Unimos las 2 tablas:

In [0]:
datos1 = pd.merge(otras_secciones_2014_imp1[["h_2014","p2_74","p1_2d_1"]]
                  ,databol5[["h_2014","renthog13_eur14","neducdom","nsitlabdom","np1","nnumadtrab"]] 
                  , how="inner", on="h_2014")
datos1.shape

### descriptivos univariantes

Descriptivo rápido de los datos:

In [0]:
datos1.describe()

In [0]:
datos1.p1_2d_1.hist(bins=20)  # edad
plt.title("Edad del CF")
plt.xlabel("años")

Transformamos la renta con el logaritmo

In [0]:
datos1["Lrenthog13_eur14"] = np.nan
datos1.loc[datos1["renthog13_eur14"]>0, "Lrenthog13_eur14"] = np.log(datos1.loc[datos1["renthog13_eur14"]>0, "renthog13_eur14"])
datos1.Lrenthog13_eur14.hist(bins=20)  # edad
plt.title("Renta familiar")
plt.xlabel("€")

In [0]:
print(datos1.neducdom.value_counts())   # ordenado por frecuencia descendente
print(datos1.neducdom.value_counts().sort_index(0))   # ordenado por código ascendente
datos1.neducdom.value_counts().plot.pie()

In [0]:
neducdomTxt = ["inferior a bachillerato", "bachillerato", "estudios universitarios"]

datos1.neducdom.value_counts().sort_index(0).plot.pie(labels=["","",""], autopct='%.1f', title="Nivel de estudios", label="")
plt.legend(loc=3, labels=neducdomTxt)

In [0]:
nsitlabdomDict = {1:"cuenta ajena", 2:"cuenta propia", 3:"jubilado", 4:"inactivo o parado"}


datos1["nsitlabdom_txt"] = datos1["nsitlabdom"].apply(lambda x:nsitlabdomDict[x])
datos1.nsitlabdom_txt.value_counts().sort_index(0).plot.pie(autopct='%.1f', title="Nivel de estudios")
#plt.legend(loc=3, labels=neducdomTxt)


Añadir variables:
- número de miembros del hogar (B.np1): toma valores 1 a 5; uno, dos, tres, cuatro, cinco o más
- niembros trabajando (B.nnumadtrab): toma valores 0 a 3; ninguno, uno, dos, tres o más

In [0]:
# univariante: np1


In [0]:
# univariante:  nnumadtrab

Generamos la variable binaria de si compra o no un automóvil.

In [0]:
datos1["auto"] = datos1["p2_74"] > 0
datos1.auto.value_counts()

In [0]:
datos1["auto"] = np.where(datos1["p2_74"] > 0 ,1,0)

datos1.auto.value_counts()

In [0]:
autoTxt = ["No", "Si"]

print(datos1.auto.value_counts().sort_index(0))

datos1.auto.value_counts().sort_index(0).plot.pie(labels=["",""], autopct='%.1f', title="Ha comprado un automóvil el último año", label="")
plt.legend(loc=3, labels=autoTxt)

### descriptivos bivariantes


In [0]:
datos1["edad_r"] = pd.cut(datos1.p1_2d_1,7)
res1 = pd.concat([datos1["edad_r"].value_counts().sort_index(0)
           ,datos1.groupby(['edad_r'])['auto'].mean()],axis=1)

res1

In [0]:

fig = plt.figure() # Create matplotlib figure

ax = fig.add_subplot(111) # Create matplotlib axes
ax2 = ax.twinx() # Create another axes that shares the same x-axis as ax.

width = 0.4

res1.edad_r.plot(kind='bar', color='red', ax=ax, width=width, position=1)
res1.auto.plot( color='blue', ax=ax2, marker='o', ls='-', title="bivariante Edad * % Auto")

ax.set_ylabel('N')
ax2.set_ylabel('%Auto')

plt.show()

Analizamos renta contra % compra

In [0]:
datos1["Lrenthog13_eur14_r"] = pd.cut(datos1.Lrenthog13_eur14,7)
res1 = pd.concat([datos1["Lrenthog13_eur14_r"].value_counts().sort_index(0)
           ,datos1.groupby(['Lrenthog13_eur14_r'])['auto'].mean()],axis=1)


fig = plt.figure() # Create matplotlib figure

ax = fig.add_subplot(111) # Create matplotlib axes
ax2 = ax.twinx() # Create another axes that shares the same x-axis as ax.

width = 0.4

res1.Lrenthog13_eur14_r.plot(kind='bar', color='red', ax=ax, width=width, position=1)
res1.auto.plot( color='blue', ax=ax2, marker='o', ls='-', title="bivariante Renta * % Auto")

ax.set_ylabel('N')
ax2.set_ylabel('%Auto')

plt.show()

res1

In [0]:
res1 = pd.concat([datos1["neducdom"].astype("category").value_counts().sort_index(0)
           ,datos1.groupby(['neducdom'])['auto'].mean()],axis=1)


fig = plt.figure() # Create matplotlib figure

ax = fig.add_subplot(1, 1, 1) # Create matplotlib axes
ax2 = ax.twinx() # Create another axes that shares the same x-axis as ax.

width = 0.4

res1.neducdom.plot(kind='bar', color='red', ax=ax, width=width, position=1)
res1.auto.plot( color='blue', ax=ax2, marker='o', ls='-', title="bivariante Educacion * % Auto")

ax.set_ylabel('N')
ax2.set_ylabel('%Auto')

plt.show()

res1


In [0]:
res1 = pd.concat([datos1["nsitlabdom"].astype("category").value_counts().sort_index(0)
           ,datos1.groupby(['nsitlabdom'])['auto'].mean()],axis=1)


fig = plt.figure() # Create matplotlib figure

ax = fig.add_subplot(111) # Create matplotlib axes
ax2 = ax.twinx() # Create another axes that shares the same x-axis as ax.

width = 0.4

res1.nsitlabdom.plot(kind='bar', color='red', ax=ax, width=width, position=1)
res1.auto.plot( color='blue', ax=ax2, marker='o', ls='-', title="bivariante N miembros trabajan * % Auto")

ax.set_ylabel('N')
ax2.set_ylabel('%Auto')

plt.show()

res1

Añadir bivariantes de nnumadtrab, np1

In [0]:
# bivariantes:  nnumadtrab
# bivariantes: np1


In [0]:
datos1.to_pickle("datos1_seminario.pkl")

#### autor: Ferran Carrascosa 8-6-2018.